## Plotly Dash source code

This document contains the source code for the Dashboard built for SpaceX launch prediction. 

Please **do not** run this directly in this notebook. But rather, create a separate Python environment, copy the source code and data in, install the dependencies and run. 

Dependencies: 
- Pandas 1.3.2
- Plotly 5.3.1
- Dash 2.0.0

Dataset can be found in this repo

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv").rename(columns={"Unnamed: 0": "id"})
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

launch_sites = list(spacex_df["Launch Site"].value_counts().keys())
options = [{"label": s, "value": s} for s in launch_sites]
options.append({"label": "ALL", "value": "ALL"})

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id="site-dropdown", options=options, value="ALL"),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id="payload-slider", min=0, max=10000, step=1000, value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    dash.dependencies.Output("success-pie-chart", "figure"),
    [dash.dependencies.Input("site-dropdown", "value")]
)
def on_site_select(value):
    if value == "ALL":
        df = spacex_df.loc[spacex_df["class"] == 1]
        return px.pie(df[["Launch Site", "id"]], names="Launch Site")        
    else:
        df = spacex_df.loc[spacex_df["Launch Site"] == value]
        return px.pie(df[["class", "id"]], names="class")

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    dash.dependencies.Output("success-payload-scatter-chart", "figure"),
    [dash.dependencies.Input("site-dropdown", "value"), dash.dependencies.Input("payload-slider", "value")]
)
def on_range_select(site, range):
    left = range[0]
    right = range[1]
    if site == "ALL":
        df = spacex_df
    else:
        df = spacex_df.loc[spacex_df["Launch Site"] == site]
    df = df.loc[df["Payload Mass (kg)"] >= left]
    df = df.loc[df["Payload Mass (kg)"] <= right]
    return px.scatter(data_frame=df, x="Payload Mass (kg)", y="class", color="Booster Version Category")


# Run the app
if __name__ == '__main__':
    app.run_server()


ModuleNotFoundError: No module named 'dash'